In [4]:
from FAdo.reex import *
from os.path import commonprefix

%run _Node_.ipynb
%run _skel_.ipynb

In [46]:
class eTree:
    def __init__(self,regex):
        ReExp = concat(atom("#"),concat(regex.marked(),atom("$")))
        self.NT = dict()
        self.atoms = set()
        self.full_tree = construct(ReExp,"1",self)
        self.root = self.NT.get("121")
        self.next = dict()
        self.sub_trees = dict()
        self.Color()
    
    def followList(self):
        l = dict()
        for x in self.atoms:
            f_list = set()
            for w in self.atoms:
                if w.type!=epsilon:
                    if w.follow(x):
                        f_list.add(w)
            l[x]=f_list
        return l

    def build_ta(self,a):
        if self.sub_trees.get(a) is not None:
            return self.sub_trees.get(a)
        ta_atoms = set()
        for n in self.atoms:
            if n.exp.val[0]==a:
                ta_atoms.add(n)
        ta = set()
        for x in ta_atoms:
            for y in ta_atoms:
                if x is y:
                    ta.add(x.parent)
                else:
                    ta.add(self.NT.get(commonprefix([x.path,y.path])))
        self.sub_trees[a] = ta_atoms.union(ta.union(self.find_stars(a)))
        return self.sub_trees.get(a)
        #return ta_atoms.union(ta.union(self.find_stars(a)))

    def find_stars(self,a):
        ta = set()
        for n in self.atoms:
            if n.exp.val[0] == a:
                n = n.parent
                while n != self.root:
                    if n.type is star:
                        ta.add(n)
                    n = n.parent
        return ta
    
    def has_Next(self,n,a):
        s = self.next.get(n)
        if s is None:
            return False
        for x in s:
            if x.exp.val[0] == a:
                return True
        return False
    
    def Color(self):
        for x in self.atoms:
            x.pSupFirst().parent.colors.add(x.exp.val[0])
        return
    
    def check_determinism(self,n):
        for x in n.colors:
            if not checkNode(n,x):
                return False
        l = True
        if n.left != None:
            l = self.check_determinism(n.left)
        else:
            return True
        if n.right != None:
            return (l and self.check_determinism(n.right))
        return l

In [45]:
def checkNode(n,a):
    te = n.tree
    ta = te.build_ta(a)
    f = n.FirstPos(ta)
    s = n.LSA()
    if n.right.exp.ewp():
        if te.has_Next(n,a):
            return False
        if s is None:
            return False
        elif s.FirstPos(ta) is f and s.reflexive(n.pSupLas()):
            return False
    return True
                              

In [8]:
def construct(reg_exp,path,t):
        node = Node(reg_exp,path,t)
        t.NT[node.path] = node
        if node.type is star:
            node.left = construct(node.exp.arg,path+'1',t)
        elif node.type is concat or node.type is disj:
            node.left = construct(node.exp.arg1,path+'1',t)
            node.right = construct(node.exp.arg2,path+'2',t)
        return node